In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import load_model, Sequential
import tensorflow as tf
from art.estimators.classification import KerasClassifier
from art.attacks.extraction import CopycatCNN
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense

tf.compat.v1.disable_eager_execution()

DIMENSION = 256
loaded_model = load_model('./target_model')
x_stealed_loaded = np.load('x_val.npy')
y_stealed_loaded = np.loadtxt('y_val.txt', delimiter=',')

In [ ]:
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(x_stealed_loaded, y_stealed_loaded, test_size=0.30, random_state=1)

# Ataque de extraccion

In [ ]:
classifier = KerasClassifier(loaded_model)

In [ ]:
copycat_cnn = CopycatCNN(
  batch_size_fit=16,
  batch_size_query=16,
  nb_epochs=5,
  nb_stolen=500,
  classifier=classifier
)

In [ ]:
stolen_m = Sequential()
stolen_m.add(Convolution2D(32, (3, 3), input_shape=(DIMENSION, DIMENSION, 3), activation='relu'))
stolen_m.add(MaxPooling2D(pool_size=(2, 2)))
stolen_m.add(Convolution2D(128, (3, 3), activation='relu'))
stolen_m.add(MaxPooling2D(pool_size=(2, 2)))
stolen_m.add(Flatten())
stolen_m.add(Dense(512, activation='softplus'))
stolen_m.add(Dense(25, activation='sigmoid'))

stolen_m.compile(
  optimizer="adam",
  loss="categorical_crossentropy",
  metrics=["accuracy"]
)

In [ ]:
model_stolen = KerasClassifier(model=stolen_m)

In [ ]:
stolen_classifier = copycat_cnn.extract(
  x=X_train_s, 
  y=y_train_s, 
  thieved_classifier=model_stolen
)

In [ ]:
score_original = classifier._model.evaluate(
  x=X_test_s,
  y=y_test_s
)

score_stolen = stolen_classifier._model.evaluate(
  x=X_test_s,
  y=y_test_s
)

# Comparing test losses
print(f"Original test loss: {score_original[0]:.2f} "  f"vs stolen test loss: {score_stolen[0]:.2f}")

# Comparing test accuracies
print(f"Original test accuracy: {score_original[1]:.2f} " f"vs stolen test accuracy: {score_stolen[1]:.2f}")